In [1]:
!nvidia-smi

Sat Dec 25 16:11:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('RICE-DATASET.csv')
df.head()

,Soil Moisture,Temperature,Humidity,Time,Status
0,54,22,70,21,ON
1,12,20,40,104,OFF
2,34,26,35,62,ON
3,7,44,44,93,OFF
4,50,38,23,92,OFF


In [4]:
df['Status'] = df['Status'].replace({"ON":1, "OFF":0})

In [5]:
df.head()

,Soil Moisture,Temperature,Humidity,Time,Status
0,54,22,70,21,1
1,12,20,40,104,0
2,34,26,35,62,1
3,7,44,44,93,0
4,50,38,23,92,0


In [6]:
X = df.drop(columns = ['Status'])
y = df['Status']

In [7]:
X.head()

,Soil Moisture,Temperature,Humidity,Time
0,54,22,70,21
1,12,20,40,104
2,34,26,35,62
3,7,44,44,93
4,50,38,23,92


In [8]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaler.fit(X)
# X = scaler.transform(X)

In [9]:
X.shape

(100000, 4)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.3, random_state= 32)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(70000, 4)
(30000, 4)
(70000,)
(30000,)


In [12]:
X_train = np.array(X_train, dtype='float32')
y_train = np.array(y_train)
X_test = np.array(X_test, dtype='float32')
y_test = np.array(y_test)

In [13]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)

In [14]:
X_train.shape

torch.Size([70000, 4])

In [15]:
train_ds = TensorDataset(X_train,y_train)
test_ds = TensorDataset(X_test,y_test)

In [16]:
train_ds[0:4]

(tensor([[ 5., 33., 25., 35.],
         [67.,  8., 26., 61.],
         [73., 41., 24., 42.],
         [ 7., 26., 33., 84.]]), tensor([1, 0, 1, 1]))

In [17]:
test_ds[0:4]

(tensor([[33., 27., 65., 46.],
         [55., 30., 42., 26.],
         [60., 37., 22., 49.],
         [ 2., 31., 38., 89.]]), tensor([1, 1, 1, 1]))

In [18]:
batch_size = 64
train_dataloader = DataLoader(train_ds, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=batch_size,shuffle=True)

In [19]:
for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    # print(X)
    # print(y)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 4])
Shape of y:  torch.Size([64]) torch.int64


In [20]:
from torch.nn.modules.flatten import Flatten
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(4, 600),
            nn.ReLU(),
            nn.Linear(600, 300),
            nn.ReLU(),
            nn.Linear(300, 150),
            nn.ReLU(),
            nn.Linear(150, 50),
            nn.ReLU(),
            nn.Linear(50, 2)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=600, bias=True)
    (1): ReLU()
    (2): Linear(in_features=600, out_features=300, bias=True)
    (3): ReLU()
    (4): Linear(in_features=300, out_features=150, bias=True)
    (5): ReLU()
    (6): Linear(in_features=150, out_features=50, bias=True)
    (7): ReLU()
    (8): Linear(in_features=50, out_features=2, bias=True)
  )
)


In [21]:
# # Get cpu or gpu device for training.
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"Using {device} device")

# # Define model
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()
#         self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(4, 512),
#             nn.ReLU(),
#             nn.Linear(512, 512),
#             nn.ReLU(),
#             nn.Linear(512, 2)
#         )

#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits

# model = NeuralNetwork().to(device)
# print(model)

In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [23]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.693787  [    0/70000]
loss: 0.486671  [ 6400/70000]
loss: 0.428056  [12800/70000]
loss: 0.469108  [19200/70000]
loss: 0.385991  [25600/70000]
loss: 0.514540  [32000/70000]
loss: 0.497741  [38400/70000]
loss: 0.393986  [44800/70000]
loss: 0.415700  [51200/70000]
loss: 0.609041  [57600/70000]
loss: 0.422368  [64000/70000]
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.443282 

Epoch 2
-------------------------------
loss: 0.380269  [    0/70000]
loss: 0.424348  [ 6400/70000]
loss: 0.480871  [12800/70000]
loss: 0.407102  [19200/70000]
loss: 0.396708  [25600/70000]
loss: 0.496296  [32000/70000]
loss: 0.332218  [38400/70000]
loss: 0.587825  [44800/70000]
loss: 0.426133  [51200/70000]
loss: 0.469361  [57600/70000]
loss: 0.413344  [64000/70000]
Test Error: 
 Accuracy: 78.5%, Avg loss: 0.431698 

Epoch 3
-------------------------------
loss: 0.490837  [    0/70000]
loss: 0.452256  [ 6400/70000]
loss: 0.417488  [12800/70000]
loss: 0.406878  [19200/700

In [26]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [27]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [53]:
x = X_test[56].reshape(1,4)
y = y_test[56]

In [54]:
y

tensor(0)

In [55]:
classes = [
   "ON",
   "OFF"
]

model.eval()

with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "OFF", Actual: "ON"
